In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()

In [3]:
sc = SparkContext(conf=conf)

In [4]:
matrix = sc.textFile("matrix1.txt").filter(lambda e: '\t' in e)
vector = sc.textFile("vec.txt").filter(lambda e: '\t' in e)

In [5]:
matrix.collect()

[u'1,2,3, 4\t', u'4,5,6, 5\t', u'7,8,9, 9\t']

In [6]:
m = matrix.map(lambda line: line.split(','))
nRow = matrix.count()
mat = m.flatMap(lambda parts: [(int(i), int(parts[i])) for i in range(len(parts))])
nCol = mat.count()/nRow
mat.collect()

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (0, 4),
 (1, 5),
 (2, 6),
 (3, 5),
 (0, 7),
 (1, 8),
 (2, 9),
 (3, 9)]

In [7]:
a = mat.zipWithIndex()
a.collect()

[((0, 1), 0),
 ((1, 2), 1),
 ((2, 3), 2),
 ((3, 4), 3),
 ((0, 4), 4),
 ((1, 5), 5),
 ((2, 6), 6),
 ((3, 5), 7),
 ((0, 7), 8),
 ((1, 8), 9),
 ((2, 9), 10),
 ((3, 9), 11)]

In [8]:
new_mat = a.map(lambda t: [(t[1])//nCol, t[0]])
new_mat.collect()

[[0, (0, 1)],
 [0, (1, 2)],
 [0, (2, 3)],
 [0, (3, 4)],
 [1, (0, 4)],
 [1, (1, 5)],
 [1, (2, 6)],
 [1, (3, 5)],
 [2, (0, 7)],
 [2, (1, 8)],
 [2, (2, 9)],
 [2, (3, 9)]]

In [9]:
v = vector.map(lambda line: line.split(',')).flatMap(lambda e: [(int(i), int(e[i])) for i in range(len(e))])
v.collect()

[(0, 10), (1, 11)]

In [10]:
rows = new_mat.join(v)
rows.collect()

[(0, ((0, 1), 10)),
 (0, ((1, 2), 10)),
 (0, ((2, 3), 10)),
 (0, ((3, 4), 10)),
 (1, ((0, 4), 11)),
 (1, ((1, 5), 11)),
 (1, ((2, 6), 11)),
 (1, ((3, 5), 11))]

In [11]:
rows_mult = rows.map(lambda tuple: (tuple[0], (tuple[1][0][0], tuple[1][0][1] * tuple[1][1])))
rows_mult.collect()

[(0, (0, 10)),
 (0, (1, 20)),
 (0, (2, 30)),
 (0, (3, 40)),
 (1, (0, 44)),
 (1, (1, 55)),
 (1, (2, 66)),
 (1, (3, 55))]

In [12]:
rows_val = rows_mult.map(lambda t: t[1])
rows_val.collect()

[(0, 10), (1, 20), (2, 30), (3, 40), (0, 44), (1, 55), (2, 66), (3, 55)]

In [13]:
new_v = rows_val.reduceByKey(lambda v1, v2: v1 + v2)
new_v.sortByKey().collect()

[(0, 54), (1, 75), (2, 96), (3, 95)]